In [1]:
import json
import os
import itertools
from typing import Tuple

import pandas as pd
import numpy as np
import talib as ta
from tqdm import tqdm

from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET

In [2]:
with open("settings.json") as jsonfile:
    settings = json.load(jsonfile)
    
BACKTEST_FOLDER = settings["filepaths"]["backtest_folder"]
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["optimized_ml_file"])
ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, settings["filepaths"]["analysis_file"])

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if (future["volumeUsd24h"] > settings["markets"]["min_volume_usd_24h"] and 
                future["name"] not in settings["markets"]["blacklist"]):
            markets.append(future["name"])

In [5]:
optimized_params = pd.DataFrame(columns=["Name", "Multiplier", "Lookback"])

pbar = tqdm(markets)
for market in pbar:
    pbar.set_description(market)
    df = ftx.get_historical_market_data(market, interval=settings["analysis"]["interval"], start_time=settings["analysis"]["start_time"])
    
    if len(df) < settings["analysis"]["min_data_length"]:
        continue

    optimized_result = bt.optimize_m_l(df, optimize_to="TheDfactor")
    optimized_result["Name"] = market
    optimized_params = optimized_params.append(optimized_result, ignore_index=True)

XRP-PERP: 100%|██████████| 35/35 [02:54<00:00,  4.99s/it]  


In [6]:
optimized_params

,Name,Multiplier,Lookback
0,AAVE-PERP,3,10
1,ADA-PERP,4,10
2,AXS-PERP,3,10
3,BCH-PERP,3,11
4,BNB-PERP,3,10
5,BSV-PERP,4,10
6,BTC-PERP,3,10
7,DOGE-PERP,4,10
8,DOT-PERP,3,11
9,ENJ-PERP,4,10


In [7]:
optimized_params.to_csv(OPTIMIZEDML_FILEPATH, index=False)